In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests 
from lxml import etree 

## Import the data from the CSV files
Special Codes are removed and only real Bezirke are selected.

The english names are transformed the following way: whitespaces are replaces by underscore in order to make it easier to search DBpredia. Furthermore the additions to the names (urban area, rural area) are removed.

In [ ]:
# Import data from HDI file
Population = pd.read_csv("./data/raw/OGD_f0743_VZ_HIS_GEM_4.csv", sep=";", encoding='utf-8')
Years = pd.read_csv("./data/raw/OGD_f0743_VZ_HIS_GEM_4_C-H88-0.csv", sep=";", encoding='utf-8')
Gemeinden = pd.read_csv("./data/raw/OGD_f0743_VZ_HIS_GEM_4_C-GRGEMAKT-0.csv", sep=";", encoding='utf-8')
MissingHM = pd.read_csv("./data/raw/MissingHM.csv", sep=";", encoding='UTF-16')

Gemeinden['en_name'] = Gemeinden['en_name'].astype(str)
# Remove special Codes
Gemeinden['en_name'] = Gemeinden['en_name'].str[:-6]
# Remove urban area, rural area and surroundings
Gemeinden['en_name'] = Gemeinden['en_name'].str.replace(' ', '_').str.replace('_urban_area', '').str.replace('_surroundings', '').str.replace('_rural_area', '')
# Change Col Names
Gemeinden = Gemeinden.rename(columns = {'code':'Gemeinde'})
Gemeinden = pd.DataFrame(Gemeinden)

# Only select real Bezirke
Gemeinden_work = Gemeinden[0:94]
Gemeinden_work = pd.DataFrame(Gemeinden_work['en_name'])

# Change Col name of Population
Population = Population.rename(columns = {'C-H88-0':'year'})
Population = Population.rename(columns = {'C-GRGEMAKT-0':'Gemeinde'})
Population = Population.rename(columns = {'F-ISIS-1':'Bevölkerung'})

# Change the years dataframe
Years['name'] = Years['name'].str[0:4]
Years = Years.rename(columns = {'code':'year'})

## Crawl DBpedia
Crawl DBpedia to get the altitude of the main cities per Bezirk.

In [ ]:
Gemeinden_work['hm'] = np.NaN
for index, row in Gemeinden_work.iterrows():
    search = Gemeinden_work.iloc[index,0]
    try:
        data = requests.get('http://dbpedia.org/data/%s.json' % search).json()
        data = data['http://dbpedia.org/resource/%s' % search]
        Gemeinden_work.iloc[index,1] = data['http://dbpedia.org/ontology/elevation'][0]['value']
    except: continue

## Add the missing values manually from a CSV
Some cities were not found on DBpedia (19) - these are looked up manually and are now added.

In [ ]:
# Manually fill the remaining missing hm values
# Some rural areas had to be assumed to be the same height as the main city (example Innsbruck Land got the height of Innsbruck)

Gemeinden_hm = pd.merge(Gemeinden_work, MissingHM, on='en_name', how='left')
Gemeinden_hm['hm_x'] = Gemeinden_hm['hm_x'].fillna(Gemeinden_hm['hm_y'])
Gemeinden_hm = Gemeinden_hm[['en_name','hm_x']]
Gemeinden_hm.dropna(axis=0,inplace=True)
Gemeinden_hm.columns = ['en_name_y','hm']

## Merge all the data and export it for 2011
In the end a dataframe should exist that contains the name of the Bezirk, Population in 2011 and the altitude

In [ ]:
interim = pd.merge(Population, Years, on='year', how='left')
PopGemeinde = pd.merge(interim, Gemeinden, on='Gemeinde', how='left')
PopGemeinde = pd.merge(PopGemeinde, Gemeinden_hm, on='en_name_y', how='left')
PopGemeinde = PopGemeinde[['name_x', 'en_name_y', 'Bevölkerung', 'hm']]
PopGemeinde.columns = ['Year', 'Bezirk','Population', 'altitude']
PopGemeinde.dropna(axis=0,inplace=True)
PopGemeinde.set_index('Year', inplace=True)

In [ ]:
Pop2011 = PopGemeinde.loc['2011']
Pop2011.sort_values('altitude',axis = 0, ascending = True, inplace=True)
Pop2011['CumSum'] = Pop2011.loc[:,'Population'].cumsum()
Pop2011['PercentPop'] = Pop2011.loc[:,'Population']/Pop2011.loc[:,'Population'].sum()
Pop2011['CumPercentPop'] = Pop2011.loc[:,'Population'].cumsum()/Pop2011.loc[:,'Population'].sum()
Pop2011.to_csv('./data/Population2011_altitude.csv', sep = ';')